In [ ]:
import subprocess


def convert_video_to_audio(video_path):
    audio_path = video_path.replace(".mp4", ".mp3")

    command = ["ffmpeg", "-i", video_path, "-vn", audio_path]

    subprocess.run(command)


convert_video_to_audio("./files/overwatch.mp4")

In [ ]:
from pydub import AudioSegment
import math


def split_audio(audio_path, segment_size):
    audio = AudioSegment.from_file(audio_path)
    segment_count = math.ceil(len(audio) / (segment_size * 1000))

    for i in range(segment_count):
        start_time = i * segment_size * 1000
        end_time = start_time + (segment_size * 1000)
        segment = audio[start_time:end_time]
        segment.export(
            f"./files/segments/overwatch_segment_{i}.mp3", format="mp3"
        )


split_audio("./files/overwatch.mp3", 30)

In [ ]:
import glob
from openai import OpenAI

client = OpenAI()


def transcribe_audio_files(segments_folder, destination):
    audio_files = glob.glob(f"{segments_folder}/*.mp3")
    audio_files.sort()

    for audio_file in audio_files:
        with open(audio_file, "rb") as audio_file, open(
            destination, "a"
        ) as text_file:
            transcription = client.audio.transcriptions.create(
                model="gpt-4o-transcribe",
                file=audio_file,
                response_format="text",
            )
            text_file.write(transcription + " ")


transcribe_audio_files("./files/segments", "./files/transcriptions.txt")

In [ ]:
from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(
    temperature=0.1,
)

prompt = PromptTemplate.from_template("What languages are used in this model?")

response = llm.predict("Can't you use Korean?")

print(response)

In [ ]:
from utils.chatbot_session import ChatBotSession
from utils.chat_callback_handler import ChatCallbackHandler
from langchain.memory import ConversationBufferMemory

chatbot_session = ChatBotSession("document")
chat_handler = ChatCallbackHandler(chatbot_session)

chatbot_session.set_memory(
    ConversationBufferMemory(return_messages=True, memory_key="history")
)

print(chatbot_session.load_memory(_))

2025-07-22 19:46:18.197 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-22 19:46:18.199 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-22 19:46:18.201 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-22 19:46:18.201 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


[]
